In [ ]:
from ca_lecture_hall_model import Grid, show_lecture_hall_over_time

In [ ]:
# Generate a grid for given size, density and spread threshold
size = 20
density = 0.7
spread_threshold = 0.3

g = Grid(size, density, spread_threshold)

In [ ]:
g.initialize_board()
results = g.run_simulation()

### check_percolation function testing 

In [ ]:
g.show_grid()
g.generate_gif()
if g.check_percolation():
    print("percolation done")
else:
    print("percolation not done")

### Animated visualisation testing

In [ ]:
# Generate a grid for given size, density and spread threshold
size = 20
density = 0.7
spread_threshold = 0.2
steps = 100

g = Grid(size, density, spread_threshold)
g.initialize_board()
grids = g.run_simulation(steps)

show_lecture_hall_over_time(grids)